In [7]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
import urllib.request
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text from gist and covert to lowercase
url = "https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt"

with urllib.request.urlopen(url) as response:
    raw_text = response.read()

raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  148574
Total Vocab:  46


In [6]:
pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=9186330e2f0df58d38c66633e9cc3f925b6b126d1984413ee2f94fba579ee4b8
  Stored in directory: c:\users\green\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
Note: you may need to restart the kernel to use updated packages.


In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Total Patterns:  148474
Epoch 1/50
2320/2320 [==============================] - ETA: 0s - loss: 2.7592
Epoch 1: loss improved from inf to 2.75917, saving model to weights-improvement-01-2.7592-bigger.hdf5
2320/2320 [==============================] - 1554s 668ms/step - loss: 2.7592
Epoch 2/50
2320/2320 [==============================] - ETA: 0s - loss: 2.4241
Epoch 2: loss improved from 2.75917 to 2.42414, saving model to weights-improvement-02-2.4241-bigger.hdf5
2320/2320 [==============================] - 1553s 670ms/step - loss: 2.4241
Epoch 3/50
1982/2320 [========================>.....] - ETA: 3:54 - loss: 2.2397

KeyboardInterrupt: 

In [ ]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
2320/2320 [==============================] - ETA: 0s - loss: 2.1114
Epoch 1: loss improved from inf to 2.11139, saving model to weights-improvement-01-2.1114-bigger.hdf5
2320/2320 [==============================] - 1964s 847ms/step - loss: 2.1114
Epoch 2/50
  17/2320 [..............................] - ETA: 47:38 - loss: 2.1019

In [9]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:


TypeError: sequence item 0: expected str instance, int found